In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

import os
import time

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# setting up display option to get entire data from describe in transpose
pd.set_option('display.max_rows', 500) 


In [21]:
df = pd.read_csv('train_fNxu4vz.csv')
df_test = pd.read_csv("test_fjtUOL8.csv")


In [22]:
del df['Loan_ID']
df['Loan_Amount_Requested'] = df['Loan_Amount_Requested'].str.replace(',','').astype(np.float64)

In [24]:
df.describe(include='all')

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
count,164309.00000,156938,138960,1.392070e+05,164309,164309,164309.000000,164309.000000,75930.000000,164309.000000,164309.000000,164309,164309.000000
unique,NaN,11,5,NaN,3,14,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,10+ years,Mortgage,NaN,VERIFIED - income,debt_consolidation,NaN,NaN,NaN,NaN,NaN,Male,NaN
freq,NaN,52915,70345,NaN,59421,97101,NaN,NaN,NaN,NaN,NaN,117176,NaN
mean,14349.33692,NaN,NaN,7.333116e+04,NaN,NaN,17.207189,0.781698,34.229356,11.193818,25.067665,NaN,2.158951
std,8281.86870,NaN,NaN,6.037750e+04,NaN,NaN,7.845083,1.034747,21.761180,4.991813,11.583067,NaN,0.738364
min,500.00000,NaN,NaN,4.000000e+03,NaN,NaN,0.000000,0.000000,0.000000,0.000000,2.000000,NaN,1.000000
25%,8000.00000,NaN,NaN,4.500000e+04,NaN,NaN,11.370000,0.000000,16.000000,8.000000,17.000000,NaN,2.000000
50%,12075.00000,NaN,NaN,6.300000e+04,NaN,NaN,16.840000,0.000000,31.000000,10.000000,23.000000,NaN,2.000000
75%,20000.00000,NaN,NaN,8.869750e+04,NaN,NaN,22.780000,1.000000,50.000000,14.000000,32.000000,NaN,3.000000


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164309 entries, 0 to 164308
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_Amount_Requested    164309 non-null  float64
 1   Length_Employed          156938 non-null  object 
 2   Home_Owner               138960 non-null  object 
 3   Annual_Income            139207 non-null  float64
 4   Income_Verified          164309 non-null  object 
 5   Purpose_Of_Loan          164309 non-null  object 
 6   Debt_To_Income           164309 non-null  float64
 7   Inquiries_Last_6Mo       164309 non-null  int64  
 8   Months_Since_Deliquency  75930 non-null   float64
 9   Number_Open_Accounts     164309 non-null  int64  
 10  Total_Accounts           164309 non-null  int64  
 11  Gender                   164309 non-null  object 
 12  Interest_Rate            164309 non-null  int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 16.3+ MB


In [26]:
# Handling Nan
df.isnull().sum()

Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [27]:
#Missing value imputation
df.Months_Since_Deliquency.fillna(0, inplace = True)
df.Annual_Income.fillna(0, inplace = True) # Another option is to impute mean to checck model performance
df.Home_Owner.fillna('msg2drop', inplace = True)
df.Length_Employed.fillna('msg2drop', inplace = True)

In [29]:
# %%time
# for i, feature in enumerate(df.columns):
#     try:
        
#         plt.figure(figsize=(12, 4))
#         plt.subplot(1 , 2 , 1)
#         sns.boxplot(x = 'Interest_Rate', y = feature, data = df, hue= 'Interest_Rate')

#         if np.issubdtype(df[feature].dtype, np.number):
#             plt.subplot(1 , 2 , 2)
#             sns.distplot(df[feature])

#     #     Individual images were exported and studied individudally   
#         plt.savefig('plots\image'+ feature +'.jpg')
#         plt.show()
#     except Exception as e:
#         print(e)

In [30]:
percentiles=[ 0,.25, .50,  .75, .90, .95,.99,.995, .999, 1]
dfOutlier = pd.DataFrame(columns=['feature', 'quantile', 'value', 'post_quantile_cont', 'post_quantile_pct'])

for feature in df.select_dtypes(include=np.number).columns.tolist():
    
    for pct in percentiles:
        qVal = df[feature].quantile(pct)
        qLen = len(df[df[feature] > qVal])
        qLenPct = round(qLen*100/len(df),3)
        
        dfOutlier = dfOutlier.append({'feature' : feature,'quantile': pct, 'value': qVal,
                                      'post_quantile_cont': qLen, 'post_quantile_pct': qLenPct}, ignore_index=True)

        
dfOutlier.to_csv('dfOutlier.csv')
dfOutlier


,feature,quantile,value,post_quantile_cont,post_quantile_pct
0,Loan_Amount_Requested,0,500.00,164306,99.998
1,Loan_Amount_Requested,0.25,8000.00,120554,73.370
2,Loan_Amount_Requested,0.5,12075.00,82120,49.979
3,Loan_Amount_Requested,0.75,20000.00,34565,21.037
4,Loan_Amount_Requested,0.9,26375.00,16313,9.928
5,Loan_Amount_Requested,0.95,30000.00,8205,4.994
6,Loan_Amount_Requested,0.99,35000.00,0,0.000
7,Loan_Amount_Requested,0.995,35000.00,0,0.000
8,Loan_Amount_Requested,0.999,35000.00,0,0.000
9,Loan_Amount_Requested,1,35000.00,0,0.000


In [31]:
#outlier managegment
originalCount = len(df)
df = df[df['Annual_Income'] <= 225000]
df = df[df['Months_Since_Deliquency'] <= 80]
df = df[df['Number_Open_Accounts'] <= 30]
df = df[df['Total_Accounts'] <= 68]

finalCount  = len(df)
print('Original Count -', originalCount , ', Final Count -', finalCount,  ', Dropped -', originalCount - finalCount, '(', round(finalCount/originalCount,4), '%)') 


Original Count - 164309 , Final Count - 161232 , Dropped - 3077 ( 0.9813 %)


In [34]:
df_test.shape

(109541, 12)

In [35]:
categorical_feature=[feature for feature in df.columns if len(df[feature].unique())<50 ]

for i in range(len(categorical_feature)):
    print(categorical_feature[i])
    print('-' * len(categorical_feature[i]))
    print('                                        ')
    print('Nan Count:  :',   df[categorical_feature[i]].isna().sum())
    print(df[categorical_feature[i]].value_counts())
    print('--------------------------------')
    


Length_Employed
---------------
                                        
Nan Count:  : 0
10+ years    51837
2 years      14354
3 years      12726
< 1 year     12579
5 years      10622
1 year       10243
4 years       9592
7 years       9113
6 years       8932
8 years       7762
msg2drop      7301
9 years       6171
Name: Length_Employed, dtype: int64
--------------------------------
Home_Owner
----------
                                        
Nan Count:  : 0
Mortgage    68646
Rent        55370
msg2drop    24895
Own         12262
Other          49
None           10
Name: Home_Owner, dtype: int64
--------------------------------
Income_Verified
---------------
                                        
Nan Count:  : 0
VERIFIED - income           58007
VERIFIED - income source    51904
not verified                51321
Name: Income_Verified, dtype: int64
--------------------------------
Purpose_Of_Loan
---------------
                                        
Nan Count:  : 0
debt_consolida

In [36]:
dummy_col_list = ['Length_Employed', 'Home_Owner', 'Income_Verified', 'Purpose_Of_Loan', 'Gender']
df = pd.get_dummies(df, columns=dummy_col_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161232 entries, 0 to 164308
Data columns (total 45 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Loan_Amount_Requested                     161232 non-null  float64
 1   Annual_Income                             161232 non-null  float64
 2   Debt_To_Income                            161232 non-null  float64
 3   Inquiries_Last_6Mo                        161232 non-null  int64  
 4   Months_Since_Deliquency                   161232 non-null  float64
 5   Number_Open_Accounts                      161232 non-null  int64  
 6   Total_Accounts                            161232 non-null  int64  
 7   Interest_Rate                             161232 non-null  int64  
 8   Length_Employed_1 year                    161232 non-null  uint8  
 9   Length_Employed_10+ years                 161232 non-null  uint8  
 10  Length_Employed_2 ye

In [37]:
#dropping 1 column from each category dummy variable list 
df = df.drop(['Length_Employed_msg2drop','Home_Owner_msg2drop',
              'Income_Verified_VERIFIED - income','Purpose_Of_Loan_renewable_energy','Gender_Male'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161232 entries, 0 to 164308
Data columns (total 40 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Loan_Amount_Requested                     161232 non-null  float64
 1   Annual_Income                             161232 non-null  float64
 2   Debt_To_Income                            161232 non-null  float64
 3   Inquiries_Last_6Mo                        161232 non-null  int64  
 4   Months_Since_Deliquency                   161232 non-null  float64
 5   Number_Open_Accounts                      161232 non-null  int64  
 6   Total_Accounts                            161232 non-null  int64  
 7   Interest_Rate                             161232 non-null  int64  
 8   Length_Employed_1 year                    161232 non-null  uint8  
 9   Length_Employed_10+ years                 161232 non-null  uint8  
 10  Length_Employed_2 ye

In [38]:
df.Interest_Rate.value_counts()

2    69340
3    58792
1    33100
Name: Interest_Rate, dtype: int64

In [39]:
df.to_csv("amit_eda.csv" , index = False)

In [40]:
df.shape

(161232, 40)